In [2]:
import torch
import math
from data_loader import NEFG3x3Set
from FCNN import AE
import matplotlib.pyplot as plt
import numpy as np

# Download the MNIST Dataset
dataset = NEFG3x3Set("info_dat_charge_std.csv", "data_3x3_12", "dat_charge_std",transform=True)

length = len(dataset)
train_split = math.floor(length*.7)
test_split = length - train_split

train_inds, test_inds = torch.utils.data.random_split(dataset, [train_split, test_split], generator=torch.Generator().manual_seed(42))



# Model Initialization
model = AE()

# Validation using MSE Loss function
loss_function = torch.nn.L1Loss()

# Using an Adam Optimizer with lr = 0.
optimizer = torch.optim.Adam(model.parameters(),
                             lr=1e-4)

train_data = torch.utils.data.DataLoader(dataset=train_inds, batch_size=32,
                                            shuffle=True)
test_data = torch.utils.data.DataLoader(dataset=test_inds, batch_size=32,
                                            shuffle=True)


In [ ]:
# inp, tar, _,_,_ = train_inds[12]
# inp = inp.float()
# print(inp.shape)
# rec = model(inp)
# print(rec.shape)
# ns = np.sqrt(rec.reshape(1,-1).shape[1]).astype(int)
# rec = rec.reshape((ns,ns))
# fig, ax = plt.subplots(nrows=1, figsize=(4,4))
# pos = ax.imshow(rec.detach().numpy())
# c_bar = fig.colorbar(pos, ax=ax)
# print(rec.min(), rec.max())

In [3]:
epochs = 20
outputs = []
losses = []
local_loss  = 0
txt1 = "Epoch {epoch} / "+str(epochs)
txt2 = "Average training loss: {loss}"
txt3 = "Average test loss: {loss}"

for epoch in range(epochs):
    print(txt1.format(epoch = epoch))
    for (inp, tar, _,_,_) in train_data:
        model.train()
        inp = inp.float()
        tar = tar.float()

        # Output of Autoencoder
        reconstructed, _  = model(inp)

        # Calculating the loss function
        loss = loss_function(reconstructed, tar)
        
        local_loss=local_loss+loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    
    print(txt2.format(loss = local_loss/len(train_data)))
    losses.append(local_loss/len(test_data))
    local_loss = 0
    
    for (inp, tar, _,_,_) in test_data:
        model.eval()
        
        inp = inp.float()
        tar = tar.float()
        # Output of Autoencoder
        reconstructed, _ = model(inp)

        # Calculating the loss function
        loss = loss_function(reconstructed, tar)
        local_loss+=loss

    
    print(txt3.format(loss = local_loss/len(test_data)))
    
    local_loss = 0



Epoch 0 / 20
Average training loss: 0.04109552130103111
Average test loss: 0.036476682871580124
Epoch 1 / 20
Average training loss: 0.0317150242626667
Average test loss: 0.03115907311439514
Epoch 2 / 20
Average training loss: 0.02656545117497444
Average test loss: 0.026866460219025612
Epoch 3 / 20
Average training loss: 0.025411229580640793
Average test loss: 0.02731805294752121
Epoch 4 / 20
Average training loss: 0.025338871404528618
Average test loss: 0.02668708749115467
Epoch 5 / 20
Average training loss: 0.025254447013139725
Average test loss: 0.02703728713095188
Epoch 6 / 20
Average training loss: 0.025269726291298866
Average test loss: 0.026639990508556366
Epoch 7 / 20
Average training loss: 0.02512899786233902
Average test loss: 0.026422757655382156
Epoch 8 / 20
Average training loss: 0.02506629377603531
Average test loss: 0.02729838155210018
Epoch 9 / 20
Average training loss: 0.025198962539434433
Average test loss: 0.026442229747772217
Epoch 10 / 20
Average training loss: 0.02

In [ ]:
# Defining the Plot Style
plt.style.use('fivethirtyeight')
plt.xlabel('Epochs')
plt.ylabel('Loss')
 
# Plotting the last 100 values
a = [i.detach().numpy() for i in losses]

plt.plot(a)
plt.show()

In [ ]:

# 1600
# 1500 - example of very small data
# 1000 - very small nums
# 3000 - bigger numbers
(inp, tar, _,_,_)  = dataset[1500]
inp = inp.float()

rec, _ = model(inp)
rec = rec.reshape((-1,26)).float()
fig, ax = plt.subplots(nrows=3, figsize=(4,12))
pos = ax[0].imshow(rec.detach().numpy())
c_bar = fig.colorbar(pos, ax=ax[0])
print(rec.min(), rec.max())

tar = tar.reshape((-1,26)).float()
pos = ax[1].imshow(tar.detach().numpy())
c_bar = fig.colorbar(pos, ax=ax[1])
print(tar.min(), tar.max())

inp = inp.reshape((-1,26)).float()
pos = ax[2].imshow(inp.detach().numpy())
c_bar = fig.colorbar(pos, ax=ax[2])
print(inp.min(), inp.max())

plt.show()
print("Prediction Loss:", loss_function(tar, rec))

print("Input Loss:", loss_function(tar, inp))





# fig, ax = plt.subplots()
# dif = rec-tar
# pos = ax.imshow(dif.detach().numpy())
# c_bar = fig.colorbar(pos, ax=ax)
# plt.show()

